In [4]:
from credmark.cmf.ipython import create_cmf
from credmark.cmf.types import Token, Contract, Address, Account, BlockNumber, Records, JoinType
from models.credmark.protocols.dexes.uniswap.univ3_math import tick_to_price, sqrt_price_x_96_to_price, tick_to_sqrt_price_x_96, sqrt_price_x_96_to_tick
from functools import partial

cmf_param = {
    'chain_id': 1,
    'block_number': 17_000_000,  # Remove this line to use the latest block
}

context, _model_loader = create_cmf(cmf_param)

# Setup pool


In [18]:
usdc_addr = Token("USDC").address
weth_addr = Token("WETH").address

uniswap_v3_factory = Contract('0x1F98431c8aD98523631AE4a59f267346ea31F984')

# ABI allows us to know function names and parameters, e.g., getPool(address, address, uint).
pool_addr = uniswap_v3_factory.functions.getPool(
    weth_addr.checksum, usdc_addr.checksum, 3000).call()
pool = Contract(pool_addr)

token0 = Token(pool.functions.token0().call())
token1 = Token(pool.functions.token1().call())

pool_fee = pool.functions.fee().call() / 1e6

In [30]:
def calc_fee(x, fee):
    if x > 0:
        return x * fee
    return 0

In [45]:
df_swaps = (pd.DataFrame(pool.fetch_events(
    pool.events.Swap, from_block=17_000_000 - 2000, to_block=17_000_000))
    .loc[:, ["blockNumber", "transactionIndex", "logIndex", "amount0", "amount1", "tick", "sqrtPriceX96"]]
    .sort_values(["blockNumber", "transactionIndex", "logIndex"])
    .assign(token_price0=lambda df: df.sqrtPriceX96.apply(partial(sqrt_price_x_96_to_price, token0_decimals=token0.decimals, token1_decimals=token1.decimals)),
            token_price1=lambda df: 1 / df.token0_price,
            amount_scaled0=lambda df: df.amount0.apply(token0.scaled),
            amount_scaled1=lambda df: df.amount1.apply(token1.scaled),

            # When a token is swapped in, only ((1 - fee) * amount) was used, (fee * amount) is the fee
            fee0=lambda df: df.amount_scaled0.apply(
                partial(calc_fee, fee=pool_fee)),
            fee1=lambda df: df.amount_scaled1.apply(
                partial(calc_fee, fee=pool_fee)),

            # when a token is swap out, we calculate the swap price
            swap_price0=lambda df, pool_fee=pool_fee: df.apply(
                lambda r: - r.amount1_scaled * (1 - pool_fee) / r.amount_scaled0 if r.amount0_scaled < 0 else np.nan, axis=1),
            swap_price1=lambda df, pool_fee=pool_fee: df.apply(
                lambda r: - r.amount0_scaled * (1 - pool_fee) / r.amount_scaled1 if r.amount1_scaled < 0 else np.nan, axis=1),

            # let's do when amount0 is negative, it means pool is selling token0, we copy the trade
            copy0=lambda df: df.apply(
                lambda r: r.amount0_scaled * (r.token0_price - r.swap_price0) if r.amount0_scaled < 0 else r.amount1_scaled, axis=1),
            ))

AttributeError: 'DataFrame' object has no attribute 'token0_price'

[1837968306931908881359176241206735, 201046, 435, 1440, 1440, 0, True]


(1858.3165550010322, (0.0005381667590700676, 1858.160102136303))

In [35]:
# When it is a negative, add the fees to it.
token0.scaled(37051716), token1.scaled(20000000000000000),

# (37.162871147999994 / 0.02) - 1858.160102136303, (37.051716 / 0.02) * (1 + 0.003) - 1858.160102136303


(-0.016544736303330865, -0.016544736303330865)

# Suppport


In [10]:
# price from SqrtPriceX96 is more precise, price from tick loses precision

end_price_from_tick = 1 / (tick_to_price(201046) *
                           10 ** (token0.decimals - token1.decimals))
end_price_from_sqrt = sqrt_price_x_96_to_price(
    1837968306931908881359176241206735, token0.decimals, token1.decimals)

end_price_from_tick, end_price_from_sqrt


(1858.3165550010322, 0.0005381667590700676)

In [11]:
# verified that it's post-trade block's sqrtPriceX96 / tick
with context.fork(block_number=16998134) as past_context:
    print(pool.functions.slot0().call())

[1837968306931908881359176241206735, 201046, 435, 1440, 1440, 0, True]


In [42]:
df_swaps

,blockNumber,transactionIndex,logIndex,amount0,amount1,tick,sqrtPriceX96,token0_price,token1_price,amount0_scaled,amount1_scaled,fee0,fee1,swap_price0,swap_price1,copy_trade
0,16998134,3,18,-37051716,20000000000000000,201046,1837968306931908881359176241206735,0.000538,1858.160102,-37.051716,0.020000,0.000000,0.000060,0.000538,NaN,-37.051716
1,16998137,15,25,-37051708,20000000000000000,201046,1837968501963859327493717608189896,0.000538,1858.159708,-37.051708,0.020000,0.000000,0.000060,0.000538,NaN,-37.051708
2,16998140,1,12,-37051700,20000000000000000,201046,1837968696995809773628258975173057,0.000538,1858.159313,-37.051700,0.020000,0.000000,0.000060,0.000538,NaN,-37.051700
3,16998187,120,186,189586197,-101722890127029920,201046,1837967702050289889351022575540184,0.000538,1858.161325,189.586197,-0.101723,0.568759,0.000000,NaN,1858.160323,-0.101723
4,16998446,68,136,-1354917920,731368239688794489,201046,1837974832344646125046288274325123,0.000538,1858.146908,-1354.917920,0.731368,0.000000,0.002194,0.000538,NaN,-1354.917920
5,16998526,134,309,-1111539942,600000000000000000,201046,1837980681896734361513883978523041,0.000538,1858.135081,-1111.539942,0.600000,0.000000,0.001800,0.000538,NaN,-1111.539942
6,16998696,18,74,1417825000,-760744371347741182,201046,1837973242890004118588110332603103,0.000538,1858.150122,1417.825000,-0.760744,4.253475,0.000000,NaN,1858.142601,-0.760744
7,16998789,3,27,-2480693810,1339060783566171874,201047,1837986297733008760356866161704559,0.000538,1858.123726,-2480.693810,1.339061,0.000000,0.004017,0.000538,NaN,-2480.693810
8,16998862,0,4,-37050983,20000000000000000,201047,1837986492718078368239119351844489,0.000538,1858.123332,-37.050983,0.020000,0.000000,0.000060,0.000538,NaN,-37.050983
9,16998867,4,37,-37050975,20000000000000000,201047,1837986687703147976121372541984419,0.000538,1858.122937,-37.050975,0.020000,0.000000,0.000060,0.000538,NaN,-37.050975
